# Solution based on AST

In [132]:
from collections import Counter

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tree_sitter_python as tspython
from sklearn.metrics import (
    f1_score,
    mean_absolute_error,
    mean_squared_error,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from tree_sitter import Language, Parser

PY_LANGUAGE = Language(tspython.language())
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

## Utils

In [133]:
def set_seed(seed: int = 420):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

## Data Loading

In [134]:
set_seed()
df = pd.read_csv("../../data/generated/dataset_dist.csv")

print(f"Total size: {len(df)}\n")

train_df, val_prep = train_test_split(df, test_size=0.3, stratify=df["generated"])
valid_df, test_df = train_test_split(
    val_prep, test_size=0.3, stratify=val_prep["generated"]
)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(valid_df)}")
print(f"Test size: {len(test_df)}")

Total size: 12428

Train size: 8699
Validation size: 2610
Test size: 1119


Check that the data is balanced

In [135]:
print(f"Mean generated (train): {train_df['generated'].mean()}")
print(f"Mean generated (validation): {valid_df['generated'].mean()}")
print(f"Mean generated (test): {test_df['generated'].mean()}")

Mean generated (train): 0.5126688125071848
Mean generated (validation): 0.512284674329502
Mean generated (test): 0.5121599642537981


## Dataset building

In [136]:
parser = Parser(PY_LANGUAGE)
node_types = set()


def walk_tree(node, types):
    types.append(node.type)
    for child in node.children:
        walk_tree(child, types)


def code_to_feature_vector(code: bytes, device=DEVICE) -> torch.Tensor:
    tree = parser.parse(code)
    types = []
    walk_tree(tree.root_node, types)
    counts = Counter(types)
    feature_vector = [counts.get(typ, 0) for typ in node_types]
    return torch.tensor(feature_vector, dtype=torch.float32, device=device)


# Gather all node types
for _, row in train_df.iterrows():
    tree = parser.parse(str.encode(row["code"]))
    types = []
    walk_tree(tree.root_node, types)
    node_types.update(types)

node_types = sorted(node_types)
type_to_idx = {typ: i for i, typ in enumerate(node_types)}

Save node types for inference

In [137]:
with open("../../data/ast/node_types.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(node_types))

In [138]:
class ASTDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, index) -> tuple[str, torch.Tensor, float]:
        raw_code = self.dataframe["code"].iloc[index]
        return (
            raw_code,
            code_to_feature_vector(raw_code.encode("utf-8")),
            float(self.dataframe["generated"].iloc[index]),
        )

    def __len__(self):
        return len(self.dataframe)


data_train = ASTDataset(dataframe=train_df)
dataloader_train = DataLoader(data_train, batch_size=64)
data_val = ASTDataset(dataframe=valid_df)
dataloader_val = DataLoader(data_val, batch_size=128)
data_test = ASTDataset(dataframe=test_df)
dataloader_test = DataLoader(data_test, batch_size=128)

## Model definition

In [139]:
class AIDetector(nn.Module):
    def __init__(self, input_dim: int, hidden_dim1: int = 64, hidden_dim2: int = 32):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim2, 1),
            nn.Sigmoid(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)

## Training

In [ ]:
def compute_metrics(predictions: list[float], labels: list[float]) -> dict:
    predictions_rounded = [round(x) for x in predictions]
    labels_rounded = [round(x) for x in labels]

    return {
        "recall": recall_score(labels_rounded, predictions_rounded),
        "roc_auc": roc_auc_score(labels_rounded, predictions_rounded),
        "f1": f1_score(labels_rounded, predictions_rounded),
        "mae": mean_absolute_error(labels, predictions),
        "mse": mean_squared_error(labels, predictions),
    }


def metrics_str(metrics: dict) -> str:
    return " | ".join([f"{key.upper()}: {value:.4f}" for key, value in metrics.items()])


def train_model(
    model: nn.Module, dataloader: DataLoader, criterion, optimizer: optim.Optimizer
):
    losses = []
    model.train()
    for _, code, label in tqdm(dataloader, desc="Training"):
        code, label = code.float().to(DEVICE), label.float().to(DEVICE)
        outputs = model(code)
        outputs = outputs.squeeze()
        loss = criterion(outputs, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)


def evaluate_model(model: nn.Module, dataloader: DataLoader) -> dict:
    model.eval()
    with torch.no_grad():
        all_predictions = []
        all_truths = []
        for _, code, label in tqdm(dataloader, desc="Validation"):
            code, label = code.float().to(DEVICE), label.float().to(DEVICE)
            outputs = model(code)
            outputs = outputs.squeeze()

            all_predictions.extend(outputs.detach().cpu().numpy().tolist())
            all_truths.extend(label.detach().cpu().numpy().tolist())

        return compute_metrics(all_predictions, all_truths)


def train_eval_loop(
    model: nn.Module,
    dataloader_train: DataLoader,
    dataloader_val: DataLoader,
    criterion,
    optimizer: optim.Optimizer,
    epochs: int = 5,
    early_stopping: int = 3,
    maximize: str = "recall",
    save_path: str = "../../data/ast/best_model.pth",
):
    best_score = 0 if maximize == "recall" else float("inf")
    no_improvement = 0
    mean_losses = []
    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}/{epochs}")
        mean_loss = train_model(model, dataloader_train, criterion, optimizer)
        metrics = evaluate_model(model, dataloader_val)

        mean_losses.append(mean_loss)
        print(f"\n{metrics_str(metrics)}\n")

        score = metrics[maximize]
        if (maximize == "recall" and score > best_score) or (
            maximize == "mae" and score < best_score
        ):
            no_improvement = 0
            best_score = score
            torch.save(model.state_dict(), save_path)
        else:
            no_improvement += 1
        if no_improvement >= early_stopping:
            print("Early stopping triggered.")
            break

    return mean_losses


set_seed()
model = AIDetector(input_dim=len(node_types)).to(DEVICE)
# criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

losses = train_eval_loop(
    model,
    dataloader_train,
    dataloader_val,
    criterion,
    optimizer,
    epochs=50,
    early_stopping=5,
    maximize="mae",
)

Epoch 1/50


Validation: 100%|██████████| 21/21 [00:01<00:00, 15.79it/s]



RECALL: 0.4721 | ROC_AUC: 0.6176 | F1: 0.5587 | MAE: 0.3745 | MSE: 0.2153

Epoch 2/50


Validation: 100%|██████████| 21/21 [00:01<00:00, 15.78it/s]



RECALL: 0.6390 | ROC_AUC: 0.6615 | F1: 0.6623 | MAE: 0.3323 | MSE: 0.1977

Epoch 3/50


...


RECALL: 0.8206 | ROC_AUC: 0.7867 | F1: 0.8014 | MAE: 0.2232 | MSE: 0.1488

Epoch 49/50


Validation: 100%|██████████| 21/21 [00:01<00:00, 17.06it/s]



RECALL: 0.8213 | ROC_AUC: 0.7859 | F1: 0.8010 | MAE: 0.2249 | MSE: 0.1511

Epoch 50/50


Validation: 100%|██████████| 21/21 [00:01<00:00, 17.29it/s]


RECALL: 0.8309 | ROC_AUC: 0.7854 | F1: 0.8028 | MAE: 0.2235 | MSE: 0.1505



## Testing

In [145]:
def test_model(model: nn.Module, dataloader: DataLoader) -> tuple[pd.DataFrame, dict]:
    model.eval()
    with torch.no_grad():
        all_predictions = []
        all_truths = []
        all_codes = []
        for real_code, code, label in tqdm(dataloader, desc="Validation"):
            code, label = code.float().to(DEVICE), label.float().to(DEVICE)
            outputs = model(code)
            outputs = outputs.squeeze()

            all_predictions.extend(outputs.detach().cpu().numpy().tolist())
            all_truths.extend(label.detach().cpu().numpy().tolist())
            all_codes.extend(real_code)

        test_df = pd.DataFrame(
            {"code": all_codes, "real": all_truths, "predicted": all_predictions}
        )
        return test_df, compute_metrics(all_predictions, all_truths)

In [146]:
best_model = AIDetector(input_dim=len(node_types)).to(DEVICE)
best_model.load_state_dict(torch.load("../../data/ast/best_model.pth"))
test_df, test_metrics = test_model(best_model, dataloader_test)
print(metrics_str(test_metrics))

test_df.to_csv("../../data/ast/test_results.csv")

Validation: 100%|██████████| 9/9 [00:00<00:00, 13.53it/s]

RECALL: 0.7955 | ROC_AUC: 0.7935 | F1: 0.8003 | MAE: 0.2208 | MSE: 0.1495


## Inference

In [147]:
def detect_ai_code(code: str) -> float:
    with open("../../data/ast/node_types.txt", "r", encoding="utf-8") as f:
        node_types_loaded = f.readlines()

    loaded_model = AIDetector(input_dim=len(node_types_loaded))
    loaded_model.load_state_dict(torch.load("../../data/ast/best_model.pth"))

    code_vectorized = code_to_feature_vector(
        code.encode("utf-8"), device=torch.device("cpu")
    ).unsqueeze(0)
    with torch.no_grad():
        prediction = loaded_model(code_vectorized).squeeze().cpu().item()
    return prediction

In [148]:
code1 = """
a,b = map(int, input().split())
if a > b:
    return 1
return 0
"""

code2 = """
x, y = map(int, input().split())
return int(x>y)
"""

code3 = """
l = map(int, input().split())
if l[0] > l[1] :
    return 1
else:
    return 0
"""
for c in [code1, code2, code3]:
    print(f"{'*' * 15}\nCODE:\n{c}\nPREDICTION: {detect_ai_code(c):.4f}\n")

***************
CODE:

a,b = map(int, input().split())
if a > b:
    return 1
return 0

PREDICTION: 0.5264

***************
CODE:

x, y = map(int, input().split())
return int(x>y)

PREDICTION: 0.9335

***************
CODE:

l = map(int, input().split())
if l[0] > l[1] :
    return 1
else:
    return 0

PREDICTION: 0.9611

